In [3]:
import geopy.distance
import pandas as pd
import os
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler, Sampler

In [4]:
def get_data(ndays):
    path_in = 'dataset_ml/sondes_in_ml_50.csv'
    df_inside = pd.read_csv(path_in, sep=',')

    path_out = 'dataset_ml/sondes_year_before_after_ml_50.csv'
    df_outside = pd.read_csv(path_out, sep=',')
    print(df_inside.id.unique().shape[0],df_outside.id.unique().shape[0])

    df_inside.nday = df_inside.nday.astype('int')
    df_inside = df_inside[(df_inside.nday < ndays)]

    df_outside.nday = df_outside.nday.astype('int')
    df_outside = df_outside[(df_outside.nday < ndays)]

    # join data per hour
    means = df_inside.groupby(['id', 'date', 'nday', 'h']).mean()
    df_inside = means.reset_index()

    means = df_outside.groupby(['id', 'date', 'nday', 'h']).mean()
    df_outside = means.reset_index()
    
    df_inside = df_inside.groupby('id').filter(lambda x: x['D'].count() == 24*ndays)
    df_inside.reset_index(drop=True, inplace=True)
    # dfnew.id.unique().shape

    df_outside = df_outside.groupby('id').filter(lambda x: x['D'].count() == 24*ndays)
    df_outside.reset_index(drop=True, inplace=True)
    
    df_inside['res'] = 1
    df_outside['res'] = 0
    df_inside.reset_index(drop=True, inplace=True)
    df_outside.reset_index(drop=True, inplace=True)
    ds = pd.concat([df_inside, df_outside], sort=True)
    ds = ds.drop(columns=['m', 'date'])
    
    print('dataset size:', ds.id.unique().shape, ds.shape)
    print('negative samples: %f, positive samples: %f'% (df_inside.id.unique().shape[0], df_outside.id.unique().shape[0]))
    return ds
    

def remove_feature(ds, feature):
    return ds.drop(columns = [feature])

def ds_to_tensor(ds):
    grouped_by_id = ds.groupby(['id', 'nday',])
    X = []
    y = []
    for name, g in grouped_by_id:
        X.append(g.drop(columns=['id', 'nday', 'h']).to_numpy())
        y.append(g.res.iloc[0])
#     print('len(X), len(y)', len(X), len(y))
    X = torch.Tensor(X)
    y = np.asarray(y, dtype=np.float32)
    return X, y

class Dataset(data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        # Select sample
        X = self.X[index]
        y = self.y[index]
        return X, y


In [5]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, 3)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        sample = []
        for eq in x:
            inner_sample = []
            for h in eq:
                inner_sample.append(self.linear1(h.type(torch.FloatTensor)))
            sample.append(torch.cat(inner_sample, dim = 0))
        y_pred = self.linear2(torch.stack(sample, dim = 0))
        return y_pred
    

def split_data(dataset, batch_size):
    data_size = dataset.__len__()
#     print('data_size =', data_size)
    validation_split = .2
    split = int(np.floor(validation_split * data_size))
    indices = list(range(data_size))
    np.random.shuffle(indices)

    train_indices, val_indices = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                             sampler=val_sampler)
    return train_loader, val_loader

In [25]:
def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, lr_scheduler = None):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
            prediction = model(x) 
            loss_value = loss(prediction, y.long())
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y)
            total_samples += y.shape[0]
#             loss_accum += loss_value
            
        if lr_scheduler: 
            lr_scheduler.step()

#         ave_loss = loss_accum / (i_step + 1)
        train_accuracy = float(correct_samples) / total_samples
        print('correct_samples=', correct_samples, 'total_samples =', total_samples)
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(loss_accum))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("loss: %f, Train accuracy: %f, Val accuracy: %f" % (loss_value, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() 
    correct = 0
    total = 0
    
    for x, y in loader:
        batch_pred = model(x).argmax(1)
        correct += (batch_pred == y).nonzero().size(0)
        total += y.size(0)     
    acc = correct / total
    return acc


In [36]:

def ablation_test(ds, ndays, features, n_tests, n_epoch, batch_size = 64):
    res = {}
    for feature in features:
        print(feature)
        print("====================================================")
        if feature == '':
            ds_new = ds
            D_in = 15
        else:
            ds_new = remove_feature(ds, feature)
            D_in = 14
        X, y = ds_to_tensor(ds_new)
        dataset = Dataset(X, y)
        print(dataset.__len__())
        feature_acc = 0
        for i_step in range(n_tests):
            print('n test =', i_step)
            train_loader, val_loader = split_data(dataset, batch_size = batch_size)
            
#             D_in, H, D_out = 14, 72*ndays, 2
            H, D_out = 72*ndays, 2
            my_model = TwoLayerNet(D_in, H, D_out)
            my_model.type(torch.FloatTensor)

            loss = nn.CrossEntropyLoss().type(torch.FloatTensor)
            optimizer = optim.Adam(my_model.parameters(), lr=0.001)
            # lr_schededuler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
            
            loss_history, train_history, val_history = train_model(my_model, train_loader, val_loader, loss, optimizer, n_epoch)
            feature_acc += max(val_history)
        res[feature] = feature_acc/n_tests
        print("====================================================")
        print('without ', feature, 'accuracy=', res[feature])
    return res
        
        


In [35]:
ds = get_data(ndays = 1)
features = ['', 'D', 'M(D)', 'MUF(D)', 'fmin', 'fminF', 'foEp', 'foEs', 'foF2', 'foF2p', 'fxI', "h'Es", "h'F", "h'F2"]
ds_new = remove_feature(ds, 'D')
X, y = ds_to_tensor(ds_new)
dataset = Dataset(X, y)
dataset.__len__()
print(y.shape)

732 1676
dataset size: (1868,) (44832, 18)
negative samples: 547.000000, positive samples: 1321.000000
(1868,)


In [37]:
res = ablation_test(ds, 1, features, n_tests=10, n_epoch = 10)
res


1868
n test = 0
correct_samples= tensor(415) total_samples = 1495
loss: 346.830170, Train accuracy: 0.277592, Val accuracy: 0.340483
correct_samples= tensor(420) total_samples = 1495
loss: 203.594254, Train accuracy: 0.280936, Val accuracy: 0.340483
correct_samples= tensor(420) total_samples = 1495
loss: 144.255325, Train accuracy: 0.280936, Val accuracy: 0.345845
correct_samples= tensor(428) total_samples = 1495
loss: 40.552055, Train accuracy: 0.286288, Val accuracy: 0.353887
correct_samples= tensor(842) total_samples = 1495
loss: 3.931520, Train accuracy: 0.563211, Val accuracy: 0.667560
correct_samples= tensor(969) total_samples = 1495
loss: 1.437589, Train accuracy: 0.648161, Val accuracy: 0.512064
correct_samples= tensor(875) total_samples = 1495
loss: 3.161456, Train accuracy: 0.585284, Val accuracy: 0.573727
correct_samples= tensor(877) total_samples = 1495
loss: 1.567957, Train accuracy: 0.586622, Val accuracy: 0.538874
correct_samples= tensor(871) total_samples = 1495
loss: 

correct_samples= tensor(1013) total_samples = 1495
loss: 73.302628, Train accuracy: 0.677592, Val accuracy: 0.739946
correct_samples= tensor(1056) total_samples = 1495
loss: 96.100998, Train accuracy: 0.706355, Val accuracy: 0.739946
correct_samples= tensor(1056) total_samples = 1495
loss: 1230926976.000000, Train accuracy: 0.706355, Val accuracy: 0.739946
correct_samples= tensor(1040) total_samples = 1495
loss: 2.427389, Train accuracy: 0.695652, Val accuracy: 0.525469
correct_samples= tensor(786) total_samples = 1495
loss: 4.893768, Train accuracy: 0.525753, Val accuracy: 0.640751
correct_samples= tensor(790) total_samples = 1495
loss: 4.086090, Train accuracy: 0.528428, Val accuracy: 0.608579
correct_samples= tensor(806) total_samples = 1495
loss: 4.449542, Train accuracy: 0.539130, Val accuracy: 0.630027
correct_samples= tensor(807) total_samples = 1495
loss: 4.787514, Train accuracy: 0.539799, Val accuracy: 0.595174
correct_samples= tensor(790) total_samples = 1495
loss: 7.940453,

loss: 2.223507, Train accuracy: 0.580602, Val accuracy: 0.573727
n test = 4
correct_samples= tensor(873) total_samples = 1495
loss: 1.899827, Train accuracy: 0.583946, Val accuracy: 0.584450
correct_samples= tensor(1050) total_samples = 1495
loss: 7.707613, Train accuracy: 0.702341, Val accuracy: 0.689008
correct_samples= tensor(1070) total_samples = 1495
loss: 2.953594, Train accuracy: 0.715719, Val accuracy: 0.664879
correct_samples= tensor(919) total_samples = 1495
loss: 1.397250, Train accuracy: 0.614716, Val accuracy: 0.557641
correct_samples= tensor(875) total_samples = 1495
loss: 1.589878, Train accuracy: 0.585284, Val accuracy: 0.589812
correct_samples= tensor(869) total_samples = 1495
loss: 2.374015, Train accuracy: 0.581271, Val accuracy: 0.571046
correct_samples= tensor(858) total_samples = 1495
loss: 3.102885, Train accuracy: 0.573913, Val accuracy: 0.595174
correct_samples= tensor(878) total_samples = 1495
loss: 1.611428, Train accuracy: 0.587291, Val accuracy: 0.587131
co

loss: 4.079478, Train accuracy: 0.559197, Val accuracy: 0.656836
correct_samples= tensor(854) total_samples = 1495
loss: 6.662321, Train accuracy: 0.571237, Val accuracy: 0.490617
n test = 1
correct_samples= tensor(1038) total_samples = 1495
loss: 102.307419, Train accuracy: 0.694314, Val accuracy: 0.734584
correct_samples= tensor(1059) total_samples = 1495
loss: 142.869598, Train accuracy: 0.708361, Val accuracy: 0.734584
correct_samples= tensor(1059) total_samples = 1495
loss: 68.168449, Train accuracy: 0.708361, Val accuracy: 0.734584
correct_samples= tensor(1059) total_samples = 1495
loss: 129.036850, Train accuracy: 0.708361, Val accuracy: 0.726542
correct_samples= tensor(1048) total_samples = 1495
loss: 37.335232, Train accuracy: 0.701003, Val accuracy: 0.715818
correct_samples= tensor(895) total_samples = 1495
loss: 3.077884, Train accuracy: 0.598662, Val accuracy: 0.544236
correct_samples= tensor(889) total_samples = 1495
loss: 3.626529, Train accuracy: 0.594649, Val accuracy: 

correct_samples= tensor(976) total_samples = 1495
loss: 5.710626, Train accuracy: 0.652843, Val accuracy: 0.691689
n test = 8
correct_samples= tensor(508) total_samples = 1495
loss: 211.328247, Train accuracy: 0.339799, Val accuracy: 0.289544
correct_samples= tensor(439) total_samples = 1495
loss: 179.099991, Train accuracy: 0.293645, Val accuracy: 0.289544
correct_samples= tensor(440) total_samples = 1495
loss: 68.869202, Train accuracy: 0.294314, Val accuracy: 0.289544
correct_samples= tensor(443) total_samples = 1495
loss: 25.477259, Train accuracy: 0.296321, Val accuracy: 0.308311
correct_samples= tensor(750) total_samples = 1495
loss: 7.493367, Train accuracy: 0.501672, Val accuracy: 0.672922
correct_samples= tensor(922) total_samples = 1495
loss: 7.473142, Train accuracy: 0.616722, Val accuracy: 0.581769
correct_samples= tensor(865) total_samples = 1495
loss: 6.724567, Train accuracy: 0.578595, Val accuracy: 0.573727
correct_samples= tensor(850) total_samples = 1495
loss: 6.44764

...

loss: 4.441731, Train accuracy: 0.563211, Val accuracy: 0.455764
correct_samples= tensor(813) total_samples = 1495
loss: 2.617220, Train accuracy: 0.543813, Val accuracy: 0.520107
correct_samples= tensor(830) total_samples = 1495
loss: 3.778346, Train accuracy: 0.555184, Val accuracy: 0.479893
correct_samples= tensor(821) total_samples = 1495
loss: 3.759716, Train accuracy: 0.549164, Val accuracy: 0.498660
correct_samples= tensor(809) total_samples = 1495
loss: 5.160904, Train accuracy: 0.541137, Val accuracy: 0.520107
correct_samples= tensor(811) total_samples = 1495
loss: 3.024688, Train accuracy: 0.542475, Val accuracy: 0.522788
correct_samples= tensor(817) total_samples = 1495
loss: 2.135058, Train accuracy: 0.546488, Val accuracy: 0.504021
n test = 9
correct_samples= tensor(920) total_samples = 1495
loss: 92.297997, Train accuracy: 0.615385, Val accuracy: 0.737265
correct_samples= tensor(1057) total_samples = 1495
loss: 43.654041, Train accuracy: 0.707023, Val accuracy: 0.737265
c

loss: 2.077854, Train accuracy: 0.684281, Val accuracy: 0.605898
correct_samples= tensor(805) total_samples = 1495
loss: 2.393820, Train accuracy: 0.538462, Val accuracy: 0.621984
correct_samples= tensor(859) total_samples = 1495
loss: 2.096621, Train accuracy: 0.574582, Val accuracy: 0.605898
correct_samples= tensor(877) total_samples = 1495
loss: 3.672798, Train accuracy: 0.586622, Val accuracy: 0.538874
correct_samples= tensor(872) total_samples = 1495
loss: 2.818888, Train accuracy: 0.583278, Val accuracy: 0.565684
correct_samples= tensor(856) total_samples = 1495
loss: 2.130174, Train accuracy: 0.572575, Val accuracy: 0.571046
correct_samples= tensor(877) total_samples = 1495
loss: 1.541481, Train accuracy: 0.586622, Val accuracy: 0.549598
correct_samples= tensor(888) total_samples = 1495
loss: 1.417937, Train accuracy: 0.593980, Val accuracy: 0.563003
n test = 6
correct_samples= tensor(801) total_samples = 1495
loss: 50.214966, Train accuracy: 0.535786, Val accuracy: 0.273458
cor

correct_samples= tensor(893) total_samples = 1495
loss: 2.430792, Train accuracy: 0.597324, Val accuracy: 0.648794
correct_samples= tensor(878) total_samples = 1495
loss: 1.801851, Train accuracy: 0.587291, Val accuracy: 0.560322
correct_samples= tensor(894) total_samples = 1495
loss: 1.998144, Train accuracy: 0.597993, Val accuracy: 0.689008
correct_samples= tensor(901) total_samples = 1495
loss: 3.967417, Train accuracy: 0.602676, Val accuracy: 0.512064
correct_samples= tensor(895) total_samples = 1495
loss: 3.115848, Train accuracy: 0.598662, Val accuracy: 0.670241
correct_samples= tensor(886) total_samples = 1495
loss: 3.685773, Train accuracy: 0.592642, Val accuracy: 0.689008
correct_samples= tensor(870) total_samples = 1495
loss: 5.426073, Train accuracy: 0.581940, Val accuracy: 0.313673
correct_samples= tensor(861) total_samples = 1495
loss: 1.996205, Train accuracy: 0.575920, Val accuracy: 0.386059
n test = 3
correct_samples= tensor(906) total_samples = 1495
loss: 15.834119, Tr

correct_samples= tensor(857) total_samples = 1495
loss: 0.498541, Train accuracy: 0.573244, Val accuracy: 0.707775
correct_samples= tensor(912) total_samples = 1495
loss: 5.671803, Train accuracy: 0.610033, Val accuracy: 0.713137
correct_samples= tensor(905) total_samples = 1495
loss: 2.184601, Train accuracy: 0.605351, Val accuracy: 0.630027
correct_samples= tensor(891) total_samples = 1495
loss: 0.945081, Train accuracy: 0.595987, Val accuracy: 0.603217
correct_samples= tensor(916) total_samples = 1495
loss: 1.119559, Train accuracy: 0.612709, Val accuracy: 0.579088
correct_samples= tensor(913) total_samples = 1495
loss: 0.549249, Train accuracy: 0.610702, Val accuracy: 0.595174
correct_samples= tensor(916) total_samples = 1495
loss: 4.667868, Train accuracy: 0.612709, Val accuracy: 0.670241
without  h'F2 accuracy= 0.702680965147453


{'': 0.6954423592493298,
 'D': 0.5957104557640751,
 'M(D)': 0.6828418230563004,
 'MUF(D)': 0.6806970509383379,
 'fmin': 0.6879356568364612,
 'fminF': 0.6766756032171581,
 'foEp': 0.7067024128686326,
 'foEs': 0.6873994638069706,
 'foF2': 0.6801608579088473,
 'foF2p': 0.6817694369973191,
 'fxI': 0.7061662198391421,
 "h'Es": 0.6876675603217157,
 "h'F": 0.6863270777479893,
 "h'F2": 0.702680965147453}

In [38]:
res

{'': 0.6954423592493298,
 'D': 0.5957104557640751,
 'M(D)': 0.6828418230563004,
 'MUF(D)': 0.6806970509383379,
 'fmin': 0.6879356568364612,
 'fminF': 0.6766756032171581,
 'foEp': 0.7067024128686326,
 'foEs': 0.6873994638069706,
 'foF2': 0.6801608579088473,
 'foF2p': 0.6817694369973191,
 'fxI': 0.7061662198391421,
 "h'Es": 0.6876675603217157,
 "h'F": 0.6863270777479893,
 "h'F2": 0.702680965147453}

In [41]:
sorted(res.items(), key=lambda x: x[1])

[('D', 0.5957104557640751),
 ('fminF', 0.6766756032171581),
 ('foF2', 0.6801608579088473),
 ('MUF(D)', 0.6806970509383379),
 ('foF2p', 0.6817694369973191),
 ('M(D)', 0.6828418230563004),
 ("h'F", 0.6863270777479893),
 ('foEs', 0.6873994638069706),
 ("h'Es", 0.6876675603217157),
 ('fmin', 0.6879356568364612),
 ('', 0.6954423592493298),
 ("h'F2", 0.702680965147453),
 ('fxI', 0.7061662198391421),
 ('foEp', 0.7067024128686326)]